In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import time

In [96]:
df = pd.read_excel('vacancies.xlsx',sheet_name='data', index_col=0)

In [97]:
df.shape

(20169, 6)

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 20169 entries, 7559095.0 to 7377133.0
Data columns (total 6 columns):
employer_id         20146 non-null float64
vacancy_type        20146 non-null object
region_name         20146 non-null object
district_name       20146 non-null object
creation_date       20146 non-null datetime64[ns]
archivation_date    20146 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 1.1+ MB


1. В каком федеральном округе в июне доля вакансий "Стандарт плюс" относительно других типов вакансий была наибольшая? В каком наименьшая? 
2. Назовите компанию-лидера по числу размещенных вакансий в Москве. Вакансии какого типа она разместила больше всего?
3. Какой процент работодателей разместил на сайте больше 1 вакансии?
4. Найдите 10-ые процентили длительности жизни вакансий каждого типа, в днях

Уберём пустые строки

In [99]:
df = df.dropna()

In [100]:
df.head()

,employer_id,vacancy_type,region_name,district_name,creation_date,archivation_date
vacancy_id,,,,,,
7559095.0,1.0,STANDART,Вологодская область,СЗФО,2020-06-18,2020-07-18
7418926.0,2.0,STANDART PLUS,Вологодская область,СЗФО,2020-06-09,2020-07-09
7387535.0,3.0,STANDART PLUS,Вологодская область,СЗФО,2020-06-08,2020-07-08
7487656.0,4.0,STANDART,Вологодская область,СЗФО,2020-06-15,2020-07-15
7379218.0,5.0,STANDART,Кабардино-Балкарская республика,СКФО,2020-06-07,2020-07-07


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 20146 entries, 7559095.0 to 7377133.0
Data columns (total 6 columns):
employer_id         20146 non-null float64
vacancy_type        20146 non-null object
region_name         20146 non-null object
district_name       20146 non-null object
creation_date       20146 non-null datetime64[ns]
archivation_date    20146 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 1.1+ MB


***Вопрос 1***

В каком федеральном округе в июне доля вакансий "Стандарт плюс" относительно других типов вакансий была наибольшая? В каком наименьшая? 

In [102]:
pivot_1 = df\
    .query("vacancy_type != 'STANDART PLUS'") \
    .query("creation_date <= '2020-06-30' and archivation_date >= '2020-06-01'") \
    .groupby('district_name', as_index=False) \
    .aggregate({'vacancy_type':'count'}) \
    .sort_values('vacancy_type', ascending=False)

pivot_st_plus = df\
    .query("vacancy_type == 'STANDART PLUS'") \
    .query("creation_date <= '2020-06-30' and archivation_date >= '2020-06-01'") \
    .groupby('district_name', as_index=False) \
    .aggregate({'vacancy_type':'count'}) \
    .sort_values('vacancy_type', ascending=False)

pivot_1["st_plus"] = pivot_st_plus['vacancy_type']
pivot_1["ratio"] = pivot_1["st_plus"] / pivot_1["vacancy_type"]
pivot_1 .sort_values('ratio', ascending=False)

,district_name,vacancy_type,st_plus,ratio
5,УрФО,701,754,1.075606
6,ЦФО,3632,3097,0.852698
1,ПФО,1699,1414,0.832254
2,СЗФО,1202,937,0.779534
4,СФО,1294,927,0.716383
7,ЮФО,1254,745,0.594099
3,СКФО,192,101,0.526042
0,ДВФО,721,236,0.327323


***Ответ:***
- Доля вакансий "Стандарт плюс" относительно других типов вакансий была наибольшая в Уральском федеральном округе.
- Доля вакансий "Стандарт плюс" относительно других типов вакансий была наименьшей в Дальневосточном федеральном округе.

***Вопрос 2***

Назовите компанию-лидера по числу размещенных вакансий в Москве. Вакансии какого типа она разместила больше всего?

In [114]:
df = df.reset_index(drop=False)

In [120]:
pivot_3 = df\
    .query("region_name == 'Москва'") \
    .groupby('vacancy_type', as_index=False) \
    .aggregate({'vacancy_id':'count'}) \
    .sort_values('vacancy_id', ascending=False)

pivot_4 = df\
    .query("employer_id	 == 1222.0") \
    .query("region_name == 'Москва'") \
    .groupby('vacancy_type', as_index=False) \
    .aggregate({'vacancy_id':'count'}) \
    .sort_values('vacancy_id', ascending=False)

In [140]:
pivot_3.head(2)

,employer_id,vacancy_id
39,1222.0,67
221,1417.0,30


In [141]:
pivot_4

,vacancy_type,vacancy_id
0,STANDART,67


***Ответ:***
- Компания-лидер по числу размещенных вакансий в Москве - 1222
- Больше всего она разместила ваканций типа "STANDART"

***Вопрос 3***

Какой процент работодателей разместил на сайте больше 1 вакансии?

In [156]:
uniq_employer = df\
    .groupby('employer_id', as_index=False) \
    .aggregate({'vacancy_id':'count'}) \
    .sort_values('vacancy_id', ascending=False)

In [165]:
len(uniq_employer)

13791

In [160]:
uniq_employer_2_and_more_vac = uniq_employer\
    .query("vacancy_id	 > 1")

In [166]:
len(uniq_employer_2_and_more_vac)

2345

In [163]:
percentage_employers_more_one_vacancy = len(uniq_employer_2_and_more_vac) / len(uniq_employer) * 100

In [164]:
percentage_employers_more_one_vacancy

17.003843086070624

***Ответ:***
17% работодателей разместили на сайте больше 1 вакансии

***Вопрос 4***

Найдите 10-ые процентили длительности жизни вакансий каждого типа, в днях

In [270]:
df_4 = df

In [271]:
df_4['vacancy_life_time'] = df_4['archivation_date'] - df_4['creation_date']

In [275]:
df_4['vacancy_life_time'] = df_4['vacancy_life_time'].dt.days

In [300]:
df_4_standart = df_4\
        .query("vacancy_type == 'STANDART'")

In [301]:
df_4_standart['vacancy_life_time']\
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], interpolation='lower')

0.1     6
0.2    16
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    74
Name: vacancy_life_time, dtype: int64

In [290]:
df_4_premium = df_4\
        .query("vacancy_type == 'PREMIUM'")

In [299]:
df_4_premium['vacancy_life_time']\
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  interpolation='lower')

0.1    13
0.2    25
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    57
Name: vacancy_life_time, dtype: int64

In [302]:
df_4_ap = df_4\
        .query("vacancy_type == 'AP'")

In [303]:
df_4_ap['vacancy_life_time']\
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], interpolation='lower')

0.1    30
0.2    30
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    30
Name: vacancy_life_time, dtype: int64

In [304]:
df_4_sp = df_4\
        .query("vacancy_type == 'STANDART PLUS'")

In [305]:
df_4_sp['vacancy_life_time']\
    .quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], interpolation='lower')

0.1    11
0.2    28
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    60
Name: vacancy_life_time, dtype: int64

***Ответ:***

10-ые процентили длительности жизни вакансий типа "STANDART"
0.1     6
0.2    16
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    74

10-ые процентили длительности жизни вакансий типа "PREMIUM"
0.1    13
0.2    25
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    57

10-ые процентили длительности жизни вакансий типа "AP"
0.1    30
0.2    30
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    30

10-ые процентили длительности жизни вакансий типа "STANDART PLUS"
0.1    11
0.2    28
0.3    30
0.4    30
0.5    30
0.6    30
0.7    30
0.8    30
0.9    30
1.0    60